## Input data: Model outputs
This data comes from scratch/XXXXXXX/Vis/output/

It contains a folder for each skill (e.g., M2, M4, etc). Each folder contains multiple .npz files that represents the sessions. ``<video_id>``-features.npz files contain the features extracted from each video (session). 
Features contain the following keys: ``['fps', 'window_limit', 'omnivore', 'slowfast', 'avion', 'objects_conf', 'objects_bbox', 'hls_l_avg', 'window_medoid']``

### Testing

In [3]:
import numpy as np
import math

user_path = "/Users/soniacq/PTG/BBN_data/updated_models/output/R19/"
# scp -r scqXXX@greene.hpc.nyu.edu:/scratch/XXXXXXX/data/BBN/features/M2/aug_img_perc0.8_NEW-YOLO_weighted_loss .
path_original_data = user_path # + 'BBN_data/RNN/aug_img_perc0.8_NEW-YOLO_weighted_loss/'
# Load the .npy file with allow_pickle=True
features_M2_1 = np.load(path_original_data + 'R19-20-features.npz', allow_pickle=True)

# Load the .npy file with allow_pickle=True
# labels_M2_1 = np.load(path_original_data + '3_tourns_2-window_label.npz', allow_pickle=True)

In [4]:
# Get the keys and convert them to a list
keys = list(features_M2_1.keys())
# Print the keys
print(keys)
# # Get the keys and convert them to a list
# keys_labels = list(labels_M2_1.keys())
# # Print the keys
# print(keys_labels)

['fps', 'window_limit', 'omnivore', 'slowfast', 'avion', 'objects_conf', 'objects_bbox', 'hls_l_avg', 'window_medoid', 'objects_class', 'label', 'label_desc']


In [5]:
temp = ['No Step'] * features_M2_1['window_medoid'].shape[0] if features_M2_1['label'].shape[0] == 0 else features_M2_1['label']
max(np.unique(temp))

6

In [36]:
features_M2_1['window_medoid']

array([   1,   11,   28,   63,   82,  120,  139,  188,  217,  234,  255,
        324,  314,  363,  396,  429,  451,  496,  495,  539,  583,  592,
        634,  669,  673,  706,  734,  780,  818,  848,  869,  894,  914,
        982,  999, 1007, 1056, 1103, 1117, 1125, 1167, 1215, 1222, 1240,
       1292, 1324, 1355, 1381, 1386, 1442, 1468])

In [7]:
omnivore_features = features_M2_1['omnivore']
print(features_M2_1['omnivore'].shape)
print(features_M2_1['slowfast'].shape)
print(features_M2_1['avion'].shape)

# array_labels = labels_M2_1['label_desc']
# print(len(array_labels))

(51, 1024)
(51, 1600)
(51, 3806)


In [8]:
window_limit_data = features_M2_1['window_limit']
n_window_frames = window_limit_data.shape[0]-1
last_IDframe = window_limit_data[n_window_frames][1]
video_duration = round(last_IDframe/features_M2_1['fps'].item()) # Use round() to round a number to the nearest integer
print("n_window_frames:", n_window_frames)
print("last_IDframe:", last_IDframe)
print("video_duration:", video_duration)


n_window_frames: 50
last_IDframe: 1495
video_duration: 50


In [9]:
window_limit_data.shape[0]

51

## Generate session metadata

In [10]:
import os
import numpy as np
import json

def compute_video_metadata(npz_file_path):
    features = np.load(npz_file_path, allow_pickle=True)
    window_limit_data = features['window_limit']
    n_window_frames = window_limit_data.shape[0] - 1
    last_IDframe = window_limit_data[n_window_frames][1]
    label = [0] * features['window_medoid'].shape[0] if features['label'].shape[0] == 0 else features['label']
    label_desc = ['No step'] * features['window_medoid'].shape[0] if features['label_desc'].shape[0] == 0 else features['label_desc']
    fps = features['fps'].item()
    duration_seconds = round(last_IDframe / fps)
    
    
    return {
        "duration_seconds": duration_seconds,
        "numberof_window_frames": n_window_frames,
        "last_IDframe": int(last_IDframe),
        "total_labels": int(len(np.unique(label))),
        "max_id_labels": int(max(np.unique(label))),
        "unique_labels": str(np.unique(label_desc)),
        "unique_id_labels": str(np.unique(label))
    }

def generate_json(directory):
    data = {}
    
    for skill_name in os.listdir(directory):
        skill_path = os.path.join(directory, skill_name)
        if os.path.isdir(skill_path):
            skill_data = {}
            for file_name in os.listdir(skill_path):
                if file_name.endswith('-features.npz'):
                    video_id = file_name.split('-features.npz')[0]
                    file_path = os.path.join(skill_path, file_name)
                    video_metadata = compute_video_metadata(file_path)
                    skill_data[video_id] = video_metadata
            if skill_data:
                data[skill_name] = skill_data

    return data

def save_json(data, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage in a Jupyter notebook
directory = "/Users/soniacq/PTG/BBN_data/updated_models/output/"  # Replace with the path to the directory containing the skill folders
output_file = "/Users/soniacq/PTG/BBN_data/updated_models/output/sessions_metadata.json"            # Replace with the desired output file path
result = generate_json(directory)
save_json(result, output_file)
print("JSON file generated successfully.")


JSON file generated successfully.


## Generate data for each window_frame for each corresponging session

In [11]:
import os
import numpy as np
import json


def generate_detailed_json(directory):
    data = []
    
    for skill_name in os.listdir(directory):
        skill_path = os.path.join(directory, skill_name)
        if os.path.isdir(skill_path):
            for file_name in os.listdir(skill_path):
                if file_name.endswith('-features.npz'):
                    video_id = file_name.split('-features.npz')[0]
                    file_path = os.path.join(skill_path, file_name)
                    features = np.load(file_path, allow_pickle=True)
                    
                    keys = list(features.keys())
                    # Print the keys
                    hls_l_avg = features['hls_l_avg']
                    label = [0] * features['window_medoid'].shape[0] if features['label'].shape[0] == 0 else features['label']
                    label_desc = ['No step'] * features['window_medoid'].shape[0] if features['label_desc'].shape[0] == 0 else features['label_desc']
                    objects_bbox = features['objects_bbox']
                    objects_conf = features['objects_conf']
                    frame_id = features['window_medoid']
                    
                    trial_data = []
                    
                    for i in range(len(hls_l_avg)):
                        window_data = {
                            "seconds": i,
                            "hls_lightness_avg": float(hls_l_avg[i]),
                            "step": float(label[i]),
                            "step_desc": label_desc[i],
                            "objects_bbox": objects_bbox[i].tolist(),
                            "objects_conf": objects_conf[i].tolist(),
                            # "objects_conf_avg": float(np.mean(objects_conf[i]))
                            "objects_conf_avg": float(np.mean(objects_conf[i])) if len(objects_conf[i]) > 0 else -1,
                            "frame_id": str(frame_id[i])
                        }
                        trial_data.append(window_data)
                    
                    subject_data = {
                        "skill_id": skill_name,
                        "session_id": video_id,
                        "data": trial_data
                    }
                    
                    data.append(subject_data)
    
    return data

def save_json(data, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage in a Jupyter notebook
directory = "/Users/soniacq/PTG/BBN_data/updated_models/output/"  # Replace with the path to the directory containing the skill folders
output_file = "/Users/soniacq/PTG/BBN_data/updated_models/output/sessions_window_frame_data.json"            # Replace with the desired output file path
result = generate_detailed_json(directory)
save_json(result, output_file)
print("JSON file generated successfully.")


JSON file generated successfully.


### Save t-SNE resutls

#### tsne position for all sessions

In [31]:
import os
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

def load_features(file_path):
    features = np.load(file_path, allow_pickle=True)
    omnivore = features['omnivore']
    slowfast = features['slowfast']
    avion = features['avion']
    window_medoid = features['window_medoid']
    objects_class = features['objects_class']
    objects_conf = features['objects_conf']
    step_label = [0] * features['window_medoid'].shape[0] if features['label'].shape[0] == 0 else features['label']
    step_label_desc = ['No step'] * features['window_medoid'].shape[0] if features['label_desc'].shape[0] == 0 else features['label_desc']
    
    return omnivore, slowfast, avion, window_medoid, objects_class, objects_conf, step_label, step_label_desc

def apply_tsne(features, metadata):
    tsne = TSNE(n_components=2, random_state=42)
    tsne_results = tsne.fit_transform(features)
    data = []
    for i, (x, y) in enumerate(tsne_results):
        data.append({
            "id": metadata[i]['id'],
            "session_id": metadata[i]['session_id'],
            "frame_id": metadata[i]['frame_id'],
            "objects_class": metadata[i]['objects_class'],
            "objects_conf": metadata[i]['objects_conf'],
            "step_label": metadata[i]['step_label'],
            "step_label_desc": metadata[i]['step_label_desc'],
            "x": float(x),  # Convert to native Python float
            "y": float(y),  # Convert to native Python float
            "method": metadata[i]['method'],
            "session": metadata[i]['session'],
            "skill": metadata[i]['skill']
        })
    return data

def generate_csv(directory, output_file):
    all_omnivore_features = []
    all_slowfast_features = []
    all_avion_features = []
    
    omnivore_labels = []
    slowfast_labels = []
    avion_labels = []
    
    for skill_name in os.listdir(directory):
        skill_path = os.path.join(directory, skill_name)
        if os.path.isdir(skill_path):
            for file_name in os.listdir(skill_path):
                if file_name.endswith('-features.npz'):
                    video_id = file_name.split('-features.npz')[0]
                    file_path = os.path.join(skill_path, file_name)
                    omnivore, slowfast, avion, window_medoid, objects_class, objects_conf, step_label, step_label_desc = load_features(file_path)
                    
                    session_length = len(omnivore) + len(slowfast) + len(avion)
                    current_session_id = 0
                    for i in range(len(omnivore)):
                        omnivore_labels.append({"id": len(omnivore_labels), "session_id": current_session_id, "frame_id": window_medoid[i], "objects_class": objects_class[i], "objects_conf": objects_conf[i], "step_label": step_label[i],"step_label_desc": step_label_desc[i], "method": 'omnivore', "session": video_id, "skill": skill_name})
                        current_session_id += 1
                    current_session_id = 0
                    for i in range(len(slowfast)):
                        slowfast_labels.append({"id": len(slowfast_labels), "session_id": current_session_id, "frame_id": window_medoid[i], "objects_class": objects_class[i], "objects_conf": objects_conf[i], "step_label": step_label[i],"step_label_desc": step_label_desc[i], "method": 'slowfast', "session": video_id, "skill": skill_name})
                        current_session_id += 1
                    current_session_id = 0
                    for i in range(len(avion)):
                        avion_labels.append({"id": len(avion_labels), "session_id": current_session_id, "frame_id": window_medoid[i], "objects_class": objects_class[i], "objects_conf": objects_conf[i], "step_label": step_label[i],"step_label_desc": step_label_desc[i], "method": 'avion', "session": video_id, "skill": skill_name})
                        current_session_id += 1
                    
                    all_omnivore_features.append(omnivore)
                    all_slowfast_features.append(slowfast)
                    all_avion_features.append(avion)
    
    # Concatenate all features
    all_omnivore_features = np.concatenate(all_omnivore_features)
    all_slowfast_features = np.concatenate(all_slowfast_features)
    all_avion_features = np.concatenate(all_avion_features)
    
    # Apply t-SNE on the concatenated features
    omnivore_data = apply_tsne(all_omnivore_features, omnivore_labels)
    slowfast_data = apply_tsne(all_slowfast_features, slowfast_labels)
    avion_data = apply_tsne(all_avion_features, avion_labels)
    
    # Combine all data
    all_data = omnivore_data + slowfast_data + avion_data
    
    # Save to CSV
    df = pd.DataFrame(all_data)
    df.to_csv(output_file, index=False)
    print(f"CSV file generated successfully: {output_file}")
    return all_data

# Example usage in a Jupyter notebook
directory = "/Users/soniacq/PTG/BBN_data/updated_models/output_sample/"  # Replace with the path to the directory containing the skill folders
output_file = "/Users/soniacq/PTG/BBN_data/updated_models/output_sample/all_sessions_tsne_results_10.csv"  # Replace with the desired output file path
all_data = generate_csv(directory, output_file)


CSV file generated successfully: /Users/soniacq/PTG/BBN_data/updated_models/output_sample/all_sessions_tsne_results_10.csv


In [34]:
import numpy as np
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
import json

def compute_silhouette_scores(data):
    print("Hello")
    # Initialize results dictionary
    results = {}
    
    # Get unique methods
    methods = set(d['method'] for d in data)
    
    for method in methods:
        # Filter data based on the method
        method_data = [d for d in data if d['method'] == method]
        
        # Extract x, y positions and skill labels
        positions = np.array([(d['x'], d['y']) for d in method_data])
        labels = np.array([d['skill'] for d in method_data])
        print(labels)
        # Ensure there are at least two unique labels
        if len(set(labels)) > 1:
            # Compute overall silhouette score and boxplot statistics
            silhouette_values = silhouette_samples(positions, labels, metric='euclidean')
            average_score = silhouette_score(positions, labels, metric='euclidean')
            boxplot_stats = calculate_boxplot_stats(silhouette_values)
            
            # Initialize method results
            method_results = {
                'average_score': average_score,
                'boxplot': boxplot_stats,
                'skills': {}
            }
            
            # Calculate silhouette score for each skill (individual class)
            skills = set(labels)
            for skill in skills:
                # Filter data for the specific skill
                skill_data = np.array([pos for pos, lbl in zip(positions, labels) if lbl == skill])
                
                # Apply KMeans clustering to get cluster labels for skill data
                if len(skill_data) > 1:
                    kmeans = KMeans(n_clusters=2, random_state=0).fit(skill_data)
                    skill_labels = kmeans.labels_
                    
                    # Compute silhouette score for skill data
                    skill_silhouette_values = silhouette_samples(skill_data, skill_labels, metric='euclidean')
                    skill_average_score = silhouette_score(skill_data, skill_labels, metric='euclidean')
                    skill_boxplot_stats = calculate_boxplot_stats(skill_silhouette_values)
                    
                    # Store skill results
                    method_results['skills'][skill] = {
                        'average_score': skill_average_score,
                        'boxplot': skill_boxplot_stats
                    }
            
            # Store method results in the main results dictionary
            results[method] = method_results
    
    # Save results to a JSON file
    with open('/Users/soniacq/PTG/BBN_data/updated_models/output_sample/silhouette_scores.json', 'w') as outfile:
        json.dump(results, outfile, indent=4)
    
    return results

def calculate_boxplot_stats(data):
    # Sort data
    sorted_data = np.sort(data)
    
    # Calculate quartiles
    q1 = np.percentile(sorted_data, 25)
    median = np.percentile(sorted_data, 50)
    q3 = np.percentile(sorted_data, 75)
    min_value = sorted_data[0]
    max_value = sorted_data[-1]
    
    return {
        'min': min_value,
        'q1': q1,
        'median': median,
        'q3': q3,
        'max': max_value
    }

# Compute silhouette scores and save to JSON
results = compute_silhouette_scores(all_data)

results


Hello
['M1' 'M1' 'M1' ... 'R16' 'R16' 'R16']
['M1' 'M1' 'M1' ... 'R16' 'R16' 'R16']
['M1' 'M1' 'M1' ... 'R16' 'R16' 'R16']


{'slowfast': {'average_score': 0.17968376813499823,
  'boxplot': {'min': -0.7484903155364779,
   'q1': -0.1215255398771232,
   'median': 0.3017885213558181,
   'q3': 0.5220592075813169,
   'max': 0.6567077461883581},
  'skills': {'M3': {'average_score': 0.6971652553913619,
    'boxplot': {'min': 0.12099854622297176,
     'q1': 0.6276702100699263,
     'median': 0.7109426338038809,
     'q3': 0.7780003123415153,
     'max': 0.8230749474151335}},
   'M1': {'average_score': 0.44872824561176927,
    'boxplot': {'min': 0.011255101718318634,
     'q1': 0.41938478591930123,
     'median': 0.46865028763614097,
     'q3': 0.5344536851174243,
     'max': 0.5819642654360723}},
   'R18': {'average_score': 0.7244084022999321,
    'boxplot': {'min': 0.0996198945121428,
     'q1': 0.69358285330433,
     'median': 0.7525290983055526,
     'q3': 0.7873448415043276,
     'max': 0.8189267069430521}},
   'A8': {'average_score': 0.5554787629861514,
    'boxplot': {'min': 0.009179757131511358,
     'q1': 0.

In [32]:
all_data

[{'id': 0,
  'session_id': 0,
  'frame_id': 1,
  'objects_class': array([], dtype=object),
  'objects_conf': array([], dtype=object),
  'step_label': 0,
  'step_label_desc': 'No step',
  'x': 85.23606872558594,
  'y': -19.83711814880371,
  'method': 'omnivore',
  'session': 'M1-4',
  'skill': 'M1'},
 {'id': 1,
  'session_id': 1,
  'frame_id': 18,
  'objects_class': array([], dtype=object),
  'objects_conf': array([], dtype=object),
  'step_label': 0,
  'step_label_desc': 'No step',
  'x': 72.09749603271484,
  'y': -8.765467643737793,
  'method': 'omnivore',
  'session': 'M1-4',
  'skill': 'M1'},
 {'id': 2,
  'session_id': 2,
  'frame_id': 17,
  'objects_class': array([], dtype=object),
  'objects_conf': array([], dtype=object),
  'step_label': 0,
  'step_label_desc': 'No step',
  'x': 72.88751220703125,
  'y': -9.137545585632324,
  'method': 'omnivore',
  'session': 'M1-4',
  'skill': 'M1'},
 {'id': 3,
  'session_id': 3,
  'frame_id': 81,
  'objects_class': array([], dtype=object),
  '